# Projet VRP/VRPTW — Notebook final

Ce notebook regroupe l'exécution du pipeline et la visualisation des résultats.

In [ ]:
from src.main import run_pipeline
init, history = run_pipeline('A-n32-k5.vrp')
print('Coût initial:', init.cost)


In [ ]:
# TODO: tracer la courbe de convergence à partir de `history`